<center> <h1 style="background-color:seagreen; color:white" >Construindo Pipelines com Spark</h1></center>  

> + Sequencia de fases executadas em ordem, que podem ser Transformers ou Estimators
> +  Instância os objetos, mas não executa (fit e transform)
> +  Você constrói o Pipeline com a Sequencia de etapas
> + E chama o método fit do pipeline para execução


<center> <h2 style="background-color:DarkKhaki; color:white" >Inicializando o PySpark</h2></center> 

In [2]:
import findspark
import pyspark
from pyspark.sql import SparkSession

# Faz a Interafce entre o Spark e o Jupyter Notebook
findspark.init()

# Inicializando uma Sessão no Spark
spark = SparkSession.builder.appName("Pepilines").getOrCreate()

---

<center> <h1 style="background-color:seagreen; color:white" >Construindo Pipelines com Spark com Multi Layers Perceptron usando o DataSet Íris</h1></center>




<center> <h2 style="background-color:DarkKhaki; color:white" >Carregando o Cunjunto de Dados Íris</h2></center> 

In [3]:
iris = spark.read.csv("../Material_do_Curso/iris.csv", header=True, inferSchema=True, sep=",")
iris.show(5)

+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



<center> <h2 style="background-color:DarkKhaki; color:white" >Separando os dados entre conjunto de traino e teste</h2></center> 

In [4]:
iris_train, iris_test = iris.randomSplit([0.8, 0.2])


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Colocando os dados no formato aceito pelo Spark</h3></center>

In [5]:
from pyspark.ml.feature import VectorAssembler

In [6]:
vector = VectorAssembler(inputCols=["sepallength", "sepalwidth",
                                    "petallength", "petalwidth"], outputCol="independente")


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Convertendo um tipo categórico em numérico com StringIndexer</center>

In [7]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="class", outputCol="dependente")

---

<center> <h2 style="background-color:DarkKhaki; color:white" >Importação do Módulo do PySpark Para Criação do Modelo de Rde Neural Artificial</h2></center>

In [8]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4, 5, 4, 3],
                                     featuresCol="independente", labelCol="dependente")

---

<center> <h1 style="background-color:seagreen; color:white" >Criando o Pipeline</h1></center> 
<br>


In [9]:
from pyspark.ml import Pipeline

In [10]:
pipeline = Pipeline(stages=[vector, indexer, mlp])
modelo = pipeline.fit(iris_train)

23/03/05 22:22:55 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/03/05 22:22:56 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/03/05 22:22:56 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.25
23/03/05 22:22:56 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.125
23/03/05 22:22:56 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
23/03/05 22:22:56 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.25
23/03/05 22:22:56 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.125
23/03/05 22:22:56 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.0625
23/03/05 22:22:

<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Teste do Modelo usando o conjunto teste</center>

In [11]:
previsao = modelo.transform(iris_test)

In [14]:
previsao.select("independente", "dependente",  "prediction", "rawprediction", "probability").show(10)

+-----------------+----------+----------+--------------------+--------------------+
|     independente|dependente|prediction|       rawprediction|         probability|
+-----------------+----------+----------+--------------------+--------------------+
|[4.4,3.0,1.3,0.2]|       2.0|       2.0|[-97.150225521020...|[3.019226014798E-...|
|[4.7,3.2,1.3,0.2]|       2.0|       2.0|[-97.150225521020...|[3.019226014798E-...|
|[4.8,3.0,1.4,0.1]|       2.0|       2.0|[-97.150225521020...|[3.019226014798E-...|
|[4.8,3.4,1.9,0.2]|       2.0|       2.0|[-97.150225521020...|[3.019226014798E-...|
|[4.9,3.0,1.4,0.2]|       2.0|       2.0|[-97.150225521020...|[3.019226014798E-...|
|[5.0,2.3,3.3,1.0]|       1.0|       1.0|[188.319857930421...|[2.65873594407671...|
|[5.0,3.4,1.5,0.2]|       2.0|       2.0|[-97.150225521020...|[3.019226014798E-...|
|[5.1,3.4,1.5,0.2]|       2.0|       2.0|[-97.150225521020...|[3.019226014798E-...|
|[5.1,3.5,1.4,0.2]|       2.0|       2.0|[-97.150225521020...|[3.01922601479


---

<center> <h1 style="background-color:seagreen; color:white" >Avaliação do Modelo usando PySpark</h1></center> 
<br>


In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [20]:
performace = MulticlassClassificationEvaluator(labelCol="dependente",
                                               predictionCol="prediction",
                                               metricName="accuracy")
acuracia = performace.evaluate(previsao)
print(f"Acurácia do MOdelo: {acuracia}")

Acurácia do MOdelo: 0.9705882352941176
